In [ ]:
!pip install beautifulSoup4
!pip install requests
!pip install yfinance
!pip install yahoofinancials

In [ ]:
from bs4 import BeautifulSoup
import urllib.request,sys,time
import datetime
import requests
from time import sleep

import pandas as pd
import numpy as np
import csv
import matplotlib.pyplot as plt
import plotly.graph_objects as go

import yfinance as yf
from yahoofinancials import YahooFinancials

In [ ]:
#Global constants

_URL = 'https://www.prnewswire.com/news-releases/news-releases-list/'
_HOMEURL = 'https://www.prnewswire.com/'
_MONTH_VALS = {"Jan": 1, "Feb": 2, "Mar": 3, "Apr": 4, "May": 5, "Jun":6, "Jul": 7, "Aug": 8, "Sep": 9, "Oct": 10, "Nov": 11, "Dec": 12}

In [ ]:
count = 1
newUrl = ''
newsList = []

### Section 1

Scan (the last two weeks or the last week, upto you) and parse news from “newswire” websit.

In [ ]:
def scrapeData(listing):
  topics = listing.find('a')['href']
  # print('topic :' ,topics)
  link = _HOMEURL+topics
  extractNews = listing.h3
  date, time = getDateTime(listing.h3.small.text.split(","))
  # print('Date:',dateTime)
  heading = extractNews.text.split('\n')[2]
  # print(heading)
  return [heading,date,time,link]

In [ ]:
class LimitReached(Exception):
    ''' 
        Custom Exception class to pause exception on reaching limit
    '''
    pass

In [ ]:
def getDateTime(timeStamp):
    '''Helper function that return date and time of the news post'''
    
    if len(timeStamp) is not 1:
      year = int(timeStamp[1].strip())
      month = _MONTH_VALS[timeStamp[0].split(" ")[0]]
      date = int(timeStamp[0].split(" ")[1])
      # Check if the limit of 14 days has been reached or not
      if (datetime.datetime.today() - datetime.datetime(year, month, date)).days is 14:
        raise LimitReached
          # If yes raise exception to break loop
          # raise LimitReached
      date = datetime.datetime(year, month, date).strftime("%b %d, %Y")
      time = timeStamp[-1].split(" ")[-2]
    else:
      date = datetime.datetime.today().strftime("%b %d, %Y")
      time = timeStamp[0][0:-3]
    return date, time

In [ ]:
while True:
  newUrl = _URL + '?page=' + str(count) + '&pagesize=100'
  
  try:
    page=requests.get(newUrl)
  except Exception as e:  
    error_type, error_obj, error_info = sys.exc_info()  
    print ('ERROR FOR LINK:',_URL)  
    print (error_type, 'Line:', error_info.tb_lineno)
  
  soup = BeautifulSoup(page.text, "html.parser")
  news=soup.find_all('div',attrs={'class':'row arabiclistingcards'})

  try:
    for listing in news:
      newsList.append(scrapeData(listing))
  except LimitReached:
    break
  count += 1
  sleep(2)

In [ ]:
newsList

In [ ]:
header = ['News','Date','Time','Link']
with open('News.csv', 'w') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(header)
    writer.writerows(newsList)

In [ ]:
df = pd.read_csv('News.csv')
df.head()

,News,Date,Time,Link
0,Electronic Trial Master File Systems Market Si...,"Apr 12, 2022",02:45,https://www.prnewswire.com//news-releases/elec...
1,"Pet Tech Market to hit US$ 20 billion by 2028,...","Apr 12, 2022",02:32,https://www.prnewswire.com//news-releases/pet-...
2,Autoliv launches multi-year commitment to supp...,"Apr 12, 2022",02:28,https://www.prnewswire.com//news-releases/auto...
3,Concentric receives a development contract fro...,"Apr 12, 2022",02:27,https://www.prnewswire.com//news-releases/conc...
4,Loomis AB issues SEK 300 million of Sustainabi...,"Apr 12, 2022",02:23,https://www.prnewswire.com//news-releases/loom...


In [ ]:
df.dropna()

,News,Date,Time,Link
0,Electronic Trial Master File Systems Market Si...,"Apr 12, 2022",02:45,https://www.prnewswire.com//news-releases/elec...
1,"Pet Tech Market to hit US$ 20 billion by 2028,...","Apr 12, 2022",02:32,https://www.prnewswire.com//news-releases/pet-...
2,Autoliv launches multi-year commitment to supp...,"Apr 12, 2022",02:28,https://www.prnewswire.com//news-releases/auto...
3,Concentric receives a development contract fro...,"Apr 12, 2022",02:27,https://www.prnewswire.com//news-releases/conc...
4,Loomis AB issues SEK 300 million of Sustainabi...,"Apr 12, 2022",02:23,https://www.prnewswire.com//news-releases/loom...
...,...,...,...,...
987,USD 23.83 Mn growth expected in Annuloplasty R...,"Apr 11, 2022",01:45,https://www.prnewswire.com//news-releases/usd-...
988,Pet Care Market size to increase by USD 48.94 ...,"Apr 11, 2022",01:30,https://www.prnewswire.com//news-releases/pet-...
989,Realestate.com.kh Partners with Realsee to Tra...,"Apr 11, 2022",01:24,https://www.prnewswire.com//news-releases/real...
990,Industrial Energy Efficiency Services Market S...,"Apr 11, 2022",01:15,https://www.prnewswire.com//news-releases/indu...


In [ ]:
newsList = df['News']
newsList

0      Electronic Trial Master File Systems Market Si...
1      Pet Tech Market to hit US$ 20 billion by 2028,...
2      Autoliv launches multi-year commitment to supp...
3      Concentric receives a development contract fro...
4      Loomis AB issues SEK 300 million of Sustainabi...
                             ...                        
987    USD 23.83 Mn growth expected in Annuloplasty R...
988    Pet Care Market size to increase by USD 48.94 ...
989    Realestate.com.kh Partners with Realsee to Tra...
990    Industrial Energy Efficiency Services Market S...
991    Gym Management Software Market Size to Grow by...
Name: News, Length: 992, dtype: object

In [ ]:
_SYMBOLS = ['CDR','FHN','STYE','GDNSF','VOLT','TLMD']

stock_news = {}

for symb in _SYMBOLS:
    stock_news[symb] = [index for index, newsString in enumerate(newsList) if symb in newsString]

{'CDR': [], 'FHN': [], 'GDNSF': [], 'STYE': [], 'TLMD': [], 'VOLT': []}